In [1]:
import numpy as np
import matplotlib.pyplot as plt
import itertools as itr
import time
from boundaries import WallBoundary

# Initializing a rank three tensor filled with nodes
latticeSize = 9
xResolution = 5
yResolution = 5
relaxationTime = 0.5
# Weights
unitVect = np.array([
	[0, 0],	[1, 0], [0, 1],
	[-1, 0], [0, -1], [1, 1],
	[-1, 1], [-1, -1], [1, -1]
])
unitX = np.array([
	0, 1, 0, -1, 0, 1, -1, -1, 1
])
unitY = np.array([
	0, 0, 1, 0, -1, 1, 1, -1, -1
])
weight = np.array([
    4/9, 1/9, 1/9,
    1/9, 1/9, 1/36,
    1/36, 1/36, 1/36
])
cs = np.sqrt(3)

# Initializing the fluid matrix
fluid = np.ones((xResolution, yResolution, latticeSize)) + 0.1 * np.random.randn(xResolution, yResolution, latticeSize)
# Assigning a right velocity
fluid[:, :, 1] = 2.3

# Set a boundary
boundary = WallBoundary(xResolution, yResolution, False)
boundary.cylindricalWall([xResolution/2, yResolution/2], 3)

In [2]:
# One iteration only
iteratedFluid = np.ones((xResolution, yResolution, latticeSize))

# Internal collision step
density = np.sum(fluid, axis = 2)
mesoMomentum = fluid.reshape(xResolution, yResolution, latticeSize, 1) * unitVect
momentum = np.sum(mesoMomentum, axis = 2) / density.reshape(xResolution, yResolution, 1)
momentumMagnitude = np.linalg.norm(momentum, axis = 2) ** 2 / (2 * cs**4) # Represents u . u

# I kinda give up on broadcasting rn. Help me :skull:
for xIndex, yIndex in itr.product(range(xResolution), range(yResolution)):
    latticeDensity = density[xIndex, yIndex]
    cellMomentumMagnitude = momentumMagnitude[xIndex, yIndex]
    for latticeIndex in range(latticeSize):
        momentumDot = np.dot(unitVect[latticeIndex], momentum[xIndex, yIndex])
        fluidEquilibrium = latticeDensity * weight[latticeIndex] * (
			1 + momentumDot / (cs ** 2) + momentumDot ** 2 / (cs ** 4) - cellMomentumMagnitude
		)
        iteratedFluid[xIndex, yIndex, latticeIndex] = fluid[xIndex, yIndex, latticeIndex] * (1 - 1/relaxationTime) + 1/relaxationTime * fluidEquilibrium

iteratedFluid

array([[[ 8.12211654e+00,  1.42364953e-01,  1.44034403e+00,
          1.24464624e+00,  1.28369158e+00, -3.96513331e-01,
         -4.36093464e-01, -4.32705275e-01, -4.62795476e-01],
        [ 8.39275288e+00,  1.64320597e-01,  1.32672331e+00,
          1.21221314e+00,  1.40514266e+00, -5.32729463e-01,
         -4.56850962e-01, -4.51768099e-01, -4.85992757e-01],
        [ 8.09223339e+00,  7.46409002e-02,  1.31451550e+00,
          1.25886538e+00,  1.41735728e+00, -5.07000713e-01,
         -4.05459325e-01, -6.22152088e-01, -4.18368169e-01],
        [ 8.22671260e+00,  5.69587345e-02,  1.23949590e+00,
          1.03853504e+00,  1.27332704e+00, -2.85000473e-01,
         -4.99528317e-01, -3.75463505e-01, -4.31668414e-01],
        [ 8.37412539e+00,  1.29813435e-01,  1.35457196e+00,
          1.14095085e+00,  1.28747981e+00, -5.17510594e-01,
         -4.71635082e-01, -4.31701419e-01, -3.75352658e-01]],

       [[ 8.26080714e+00,  1.10234984e-01,  1.33330285e+00,
          1.26069551e+00,  1.1862

In [175]:
for latticeIndex, shiftX, shiftY in zip(range(latticeSize), unitX, unitY):
    iteratedFluid[:, :, latticeIndex] = np.roll(iteratedFluid[:, :, latticeIndex], shift = shiftX, axis = 1)
    iteratedFluid[:, :, latticeIndex] = np.roll(iteratedFluid[:, :, latticeIndex], shift = shiftY, axis = 0)

fluidBoundary = boundary.boundary
boundaryFluid = iteratedFluid[fluidBoundary, :]

boundaryFluid.

A B
C D

(24, 9)